In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import tensorflow as tf
import math
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('..')
from detector.rpn import generate_anchors
from detector.utils import prune_outside_window
from detector.training_target_creation import create_targets, match_boxes
from detector.input_pipeline import Pipeline
from params import coco_params

In [ ]:
def draw_boxes(image, boxes, color='red'):
    image_copy = image.copy()
    draw = ImageDraw.Draw(image_copy, 'RGBA')
    for box in boxes:
        ymin, xmin, ymax, xmax = box
        fill = (255, 255, 255, 10)
        outline = color
        draw.rectangle(
            [(xmin, ymin), (xmax, ymax)],
            fill=fill, outline=outline
        )
    return image_copy

# Create a graph

In [ ]:
tf.reset_default_graph()

pipeline = Pipeline(
    ['/mnt/datasets/COCO/val_shards/shard-0000.tfrecords'],
    is_training=True, params=coco_params
)
iterator = pipeline.dataset.make_one_shot_iterator()
features, labels = iterator.get_next()


stride = 32
image = features['images'][0]
image_width, image_height = tf.shape(image)[1], tf.shape(image)[0]
width, height = tf.to_int32(tf.ceil(image_width/stride)), tf.to_int32(tf.ceil(image_height/stride))


raw_anchors = generate_anchors(
    (width, height), scales=[32, 64, 128, 256, 512],
    aspect_ratios=[1.0],
    anchor_stride=(stride, stride), anchor_offset=(0, 0)
)
anchors = tf.reshape(raw_anchors, [-1, 4])
window = tf.to_float(tf.stack([0, 0, image_height - 1, image_width - 1]))
anchors, _ = prune_outside_window(anchors, window)


groundtruth_boxes = labels['boxes'][0]
groundtruth_labels  = labels['labels'][0]
N = tf.shape(groundtruth_boxes)[0]
num_anchors = tf.shape(anchors)[0]
only_background = tf.fill([num_anchors], -1)
matches = tf.cond(
    tf.greater(N, 0),
    lambda: match_boxes(
        anchors, groundtruth_boxes,
        positives_threshold=0.5,
        negatives_threshold=0.3,
        force_match_groundtruth=True
    ),
    lambda: only_background
)


anchor_matches = tf.to_int32(matches)
rpn_reg_targets, _ = create_targets(
    anchors, groundtruth_boxes,
    groundtruth_labels, anchor_matches
)

In [ ]:
with tf.Session() as sess:
    result = sess.run({
        'image': image, 
        'anchors': anchors,
        'raw_anchors': raw_anchors,
        'matches': anchor_matches, 
        'boxes': groundtruth_boxes,
        'targets': rpn_reg_targets,
    })    
matched = np.where(result['matches'] >= 0)[0]

# Draw

In [ ]:
i = Image.fromarray((255.0*result['image']).astype('uint8'))
i = draw_boxes(i, result['boxes'], color='red')
draw_boxes(i, result['anchors'][matched], color='blue')

# Show non clipped anchors

In [ ]:
ymin, xmin, ymax, xmax = [result['raw_anchors'][:, :, :, i] for i in range(4)]

h, w = (ymax - ymin), (xmax - xmin)
cy, cx = ymin + 0.5*h, xmin + 0.5*w

centers = np.stack([cy, cx], axis=3)
anchor_sizes = np.stack([h, w], axis=3)

In [ ]:
image_h, image_w, _ = result['image'].shape
fig, ax = plt.subplots(1, dpi=100, figsize=(int(5*image_w/image_h), 5))
unique_centers = centers[:, :, 0, :].reshape(-1, 2)
unique_sizes = anchor_sizes[0, 0, :, :]

i = 300
for j, point in enumerate(unique_centers):
    cy, cx = point
    color = 'g' if j == i else 'r' 
    ax.plot([cx], [cy], marker='o', markersize=3, color=color)

cy, cx = unique_centers[i] 
for box in unique_sizes:
    h, w = box
    xmin, ymin = cx - 0.5*w, cy - 0.5*h
    rect = plt.Rectangle(
        (xmin, ymin), w, h,
        linewidth=1.0, edgecolor='k', facecolor='none'
    )
    ax.add_patch(rect)

plt.xlim([0, image_w]);
plt.ylim([0, image_h]);